In [1]:
import matplotlib.pyplot as plt
from joblib import load, dump
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import warnings, os


import seaborn as sns
import scipy.stats as stats
import tensorflow as tf

import aggmap
from aggmap import loadmap, aggmodel


import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
from config import P

seed = P.seed #for the random permutation
metric = P.metric  # similarity measurement
var_thr = P.var_thr  # feature variance should larger than 0

color_list = P.color_list
mnist_labels_dict = P.fmnist_labels_dict
data_save_folder = P.fmnist_data_save_folder
results_save_folder = P.results_save_folder

In [3]:
epochs = P.epochs
batch_size = P.batch_size
lr = P.lr
conv1_kernel_size = P.conv1_kernel_size
dense_layers = P.dense_layers
dense_avf = P.dense_avf
last_avf = P.last_avf
patience = P.patience

In [4]:
## fix random seed to get repeatale results
np.random.seed(seed)
tf.compat.v1.set_random_seed(seed)
warnings.filterwarnings("ignore")

In [5]:
if not os.path.exists(results_save_folder):
    os.makedirs(results_save_folder)

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() #(x_train, y_train), (x_test, y_test)


In [7]:
Y = pd.get_dummies(y_train).values.astype(float)
testY = pd.get_dummies(y_test).values.astype(float)

In [8]:
#orignal images
X_OR1 = load(os.path.join(data_save_folder, '01_X_OR1.data'))

#random shuffle
X_ORS1 = load(os.path.join(data_save_folder, '02_X_ORS1.data'))

#agg
X_ORSAgg1 = load(os.path.join(data_save_folder, '03_X_ORSAgg1.data'))

#cluster split channel
X_ORSAggC5 = load(os.path.join(data_save_folder, '04_X_ORSAggC5.data'))

In [9]:
fmp_types = [X_OR1,  X_ORSAgg1, X_ORSAggC5] #   X_ORS1,  X_ORSAgg1, X_ORSAggC5,
fmp_names = ['Org1', 'OrgRPAgg1', 'OrgRPAgg5'] # 'OrgRP1', 'OrgRPAgg1', 'OrgRPAgg5'


In [10]:
for name, x in zip(fmp_names, fmp_types) :
    
    X, testX = x[0]/255., x[1]/255.

    trainX, validX, trainY, validY = train_test_split(X, Y, test_size = 0.1, random_state = seed)

    model = aggmodel.net.AggMapNet(trainX.shape[1:],
                                   n_outputs=trainY.shape[-1], 
                                   conv1_kernel_size = conv1_kernel_size,
                                   dense_layers=dense_layers, 
                                   dense_avf = dense_avf, 
                                   last_avf=last_avf)

    opt = tf.keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) #

    performance = aggmodel.cbks.CLA_EarlyStoppingAndPerformance((trainX, trainY), 
                                                                (validX, validY), 
                                                                patience = patience,
                                                                criteria = 'val_auc',
                                                                metric = 'ACC',
                                                                last_avf= last_avf,
                                                                verbose = 0)

    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

    history = model.fit(trainX, trainY, batch_size=batch_size,  # verbose= 2, 
                          epochs=epochs, shuffle = True, 
                          validation_data = (validX, validY), callbacks= [performance]) 

    train_loss, train_acc = model.evaluate(trainX, trainY, verbose=0)

    valid_loss, valid_acc = model.evaluate(validX, validY, verbose=0)

    test_loss, test_acc = model.evaluate(testX, testY, verbose=0)


    res = {'train_size':len(trainX),
           'valid_size': len(validX),
           'test_size': len(testX),
           'dense_layers':dense_layers,
           'dense_avf':dense_avf,
           'last_avf':last_avf, 
           'trainable_params':model.count_params(),
           'batch_size':batch_size,
           'lr':lr,
           'best_epochs_in_valid': performance.best_epoch + 1,
           
           'train_acc':train_acc,
           'valid_best_acc': valid_acc,
           'test_acc': test_acc,
           
           'train_loss':train_loss,
           'valid_best_loss': valid_loss,
           'test_loss': test_loss,}

    pd.DataFrame(history.history).to_csv(os.path.join(results_save_folder, 'training_%s.csv' % name))
    
    pd.DataFrame([res]).to_csv(os.path.join(results_save_folder, 'performance_%s.csv' % name))
    
    model.save(os.path.join(results_save_folder, 'model_%s.h5' % name))
    
    test_report = pd.DataFrame(classification_report(validY, np.round(model.predict(validX)),output_dict = True )).T
    test_report.to_csv(os.path.join(results_save_folder, 'test_report_%s.csv' % name))

Train on 54000 samples, validate on 6000 samples
Epoch 1/100
54000/54000 [==============================] - 12s 226us/sample - loss: 0.8334 - accuracy: 0.7124 - val_loss: 0.5455 - val_accuracy: 0.7900
Epoch 2/100
54000/54000 [==============================] - 8s 146us/sample - loss: 0.4800 - accuracy: 0.8239 - val_loss: 0.4328 - val_accuracy: 0.8425
Epoch 3/100
54000/54000 [==============================] - 8s 149us/sample - loss: 0.4165 - accuracy: 0.8481 - val_loss: 0.3980 - val_accuracy: 0.8588
Epoch 4/100
54000/54000 [==============================] - 8s 148us/sample - loss: 0.3774 - accuracy: 0.8648 - val_loss: 0.3550 - val_accuracy: 0.8707
Epoch 5/100
54000/54000 [==============================] - 8s 146us/sample - loss: 0.3503 - accuracy: 0.8738 - val_loss: 0.3528 - val_accuracy: 0.8707
Epoch 6/100
54000/54000 [==============================] - 8s 151us/sample - loss: 0.3312 - accuracy: 0.8800 - val_loss: 0.3194 - val_accuracy: 0.8828
Epoch 7/100
54000/54000 [===================

In [11]:
pd.DataFrame([res])

,train_size,valid_size,test_size,dense_layers,dense_avf,last_avf,trainable_params,batch_size,lr,best_epochs_in_valid,train_acc,valid_best_acc,test_acc,train_loss,valid_best_loss,test_loss
0,54000,6000,10000,"[128, 64]",relu,softmax,304906,64,0.0001,75,0.975556,0.918167,0.9115,0.073172,0.269543,0.298187


In [12]:
model = aggmodel.net.AggMapNet((28,28,5),
                               n_outputs=trainY.shape[-1], 
                               conv1_kernel_size = conv1_kernel_size,
                               dense_layers=dense_layers, 
                               dense_avf = dense_avf, 
                               last_avf=last_avf)

In [13]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, dpi = 600)

In [14]:
model.count_params()

304906